# Rapport projet — Documentation & exécution (Quarto / Jupyter)

Ce notebook Quarto/Jupyter présente un résumé exécutable du projet "mongodb_archiver" : configuration, lancement des simulateurs, vérifications (Change Streams), prétraitement, EDA et étapes pour produire un rapport HTML/PDF via Quarto. Il est organisé pour être rendu avec Quarto ou exécuté dans VSCode.

> Remarque : ce notebook contient des cellules de documentation et des cellules de code prêtes à être adaptées selon vos données et environnement local.

## Objectifs du rapport

- Documenter la structure du projet et les scripts de simulation.
- Fournir des cellules prêtes à l'emploi pour la configuration, le chargement des données, le prétraitement, l'analyse exploratoire, la modélisation et l'évaluation.
- Inclure des instructions pour rendre ce notebook via Quarto en HTML/PDF et intégrer des tests automatisés.


## 1) Configuration de l'environnement et dépendances

Installons / listons les paquets recommandés. Adaptez selon votre environnement (conda/venv/pip).

```yaml
# Exemple d'entête Quarto pour rendre ce notebook
title: "Rapport - mongodb_archiver"
format:
  html: default
  pdf: default
execute:
  echo: true
  warning: true
  error: true
```

Paquets recommandés : pandas, numpy, matplotlib, seaborn, scikit-learn, joblib, pymongo, python-dotenv, quarto (CLI pour rendu).

Note : Sur Windows, utilisez Git Bash ou WSL pour exécuter des scripts bash. Pour Quarto, installez Quarto CLI depuis https://quarto.org.


In [ ]:
# Vérification rapide des paquets (ne tente pas d'installer automatiquement)
required = ["pandas","numpy","matplotlib","seaborn","scikit-learn","joblib","pymongo","python_dotenv","quarto"]

import importlib
installed = {}
for pkg in required:
    try:
        importlib.import_module(pkg if pkg != 'python_dotenv' else 'dotenv')
        installed[pkg] = True
    except Exception:
        installed[pkg] = False

print("Vérification des paquets requis:")
for k,v in installed.items():
    print(f"  - {k}: {'OK' if v else 'MANQUANT'}")

print('\nSi des paquets sont manquants, installez-les via pip ou conda:')
print('  pip install pandas numpy matplotlib seaborn scikit-learn joblib pymongo python-dotenv')
print('Pour Quarto CLI, voir https://quarto.org/docs/get-started/')


## 2) Chargement des données

Cette section montre comment charger des jeux de données locaux (CSV/JSON) et afficher un aperçu. Le projet `mongodb_archiver` utilise MongoDB pour stocker les entités (Client, Commande, Livreur...). Ci‑dessous un exemple générique pour charger des fichiers plats si présents.

In [ ]:
# Exemple: lister les fichiers CSV/JSON à la racine et essayer de charger le premier trouvé
from pathlib import Path
import pandas as pd

root = Path('..')  # Par défaut, notebook est dans repo root
candidates = list(Path('.').glob('**/*.csv')) + list(Path('.').glob('**/*.json'))
print(f"Fichiers CSV/JSON trouvés: {len(candidates)}")
for f in candidates[:10]:
    print(' -', f)

if candidates:
    f = candidates[0]
    print(f"\nChargement d'exemple: {f}")
    try:
        if f.suffix.lower() == '.csv':
            df = pd.read_csv(f, nrows=5)
        else:
            df = pd.read_json(f, lines=True, nrows=5)
        print(df.head())
    except Exception as e:
        print('Erreur lecture:', e)
else:
    print('Aucun CSV/JSON trouvé — pour ce projet, les données sont dans MongoDB (voir section Change Streams).')


## 3) Prétraitement des données

Fonctions types pour nettoyer, encoder et normaliser. Ci‑dessous des exemples réutilisables et un pipeline scikit-learn.

In [ ]:
# Exemples de fonctions de pre-traitement (placeholders)
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


def build_categorical_pipeline():
    return Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])


def build_numeric_pipeline():
    return Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

print('Pipelines prêts — adaptez-les à vos colonnes.')


## 4) Analyse exploratoire (EDA)

Exemples d'analyses : distributions, boxplots, corrélation. Adaptez les colonnes selon votre dataset.

In [ ]:
# EDA placeholders: affichage statistique et graphiques simples
import matplotlib.pyplot as plt
import seaborn as sns

# exemple si df existe
try:
    df
    print('Colonnes:', df.columns.tolist())
    print('\nStatistiques descriptives:')
    display(df.describe(include='all'))

    # histogramme d'une colonne numérique si présente
    num_cols = df.select_dtypes(include='number').columns.tolist()
    if num_cols:
        plt.figure(figsize=(8,4))
        sns.histplot(df[num_cols[0]].dropna(), kde=True)
        plt.title(f'Distribution: {num_cols[0]}')
        plt.show()
except NameError:
    print('Aucun DataFrame `df` défini — chargez vos données dans la section 2.')


## 5) Construction et entraînement du modèle

Exemple générique: problème de classification/regression. Utilisez la cellule ci‑dessous comme squelette pour entraîner un modèle scikit‑learn.

In [ ]:
# Modelling skeleton (classification example)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Ceci est un squelette : remplacez X,y par vos variables
try:
    X, y
except NameError:
    X = None
    y = None

if X is None or y is None:
    print('Définissez X et y (features, target) avant d'exécuter la cellule de modélisation.')
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(random_state=42)
    params = {'n_estimators':[100], 'max_depth':[None]}
    grid = GridSearchCV(model, params, cv=3, n_jobs=-1)
    grid.fit(X_train, y_train)
    print('Best params:', grid.best_params_)
    preds = grid.predict(X_test)
    print(classification_report(y_test, preds))
    
    # sauver le modèle
    import joblib
    joblib.dump(grid.best_estimator_, 'models/best_model.joblib')
    print('Modèle sauvegardé sous models/best_model.joblib')


## 6) Évaluation du modèle et métriques

Calcul des métriques pertinentes et visualisations (ROC, matrice de confusion).

In [ ]:
# Evaluation placeholders: ROC, confusion matrix
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

try:
    y_test, preds
    # example ROC if predicted probabilities exist
    if hasattr(grid, 'predict_proba'):
        probs = grid.predict_proba(X_test)[:,1]
        fpr, tpr, _ = roc_curve(y_test, probs)
        plt.figure(); plt.plot(fpr,tpr); plt.title('ROC'); plt.show()
    cm = confusion_matrix(y_test, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(); plt.show()
except Exception:
    print('Exécutez d\'abord la cellule de modélisation pour obtenir y_test et preds.')


## 7) Export des résultats et génération Quarto (HTML/PDF)

Exemples de cellules Quarto et commandes pour rendre ce notebook en HTML ou PDF.

Pour rendre en HTML depuis la racine du projet (PowerShell) :

```powershell
# Rendre avec Quarto (installer Quarto CLI d'abord)
quarto render report_quarto_project.ipynb --to html
# ou pour PDF
quarto render report_quarto_project.ipynb --to pdf
```

Exporter un tableau de résultats :
```python
# df_results.to_csv('results/metrics.csv', index=False)
```

## 8) Tests unitaires et intégration continue

- Exemple rapide de test pytest pour une fonction de prétraitement.
- Exemple minimal de workflow GitHub Actions pour exécuter tests et rendre Quarto.

In [ ]:
# Exemple pytest minimal à placer sous tests/test_preprocessing.py
example_test = '''
import pandas as pd
from your_module import build_numeric_pipeline

def test_numeric_pipeline_handles_nans():
    df = pd.DataFrame({'a':[1, None, 3]})
    pipe = build_numeric_pipeline()
    res = pipe.fit_transform(df)
    assert res.shape[0] == 3
'''
print('Exemple de test pytest à sauvegarder dans tests/test_preprocessing.py :\n')
print(example_test)

# Exemple GitHub Actions (yaml) à placer sous .github/workflows/ci.yml
ci_example = '''
name: CI
on: [push, pull_request]
jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - run: pip install -r requirements.txt
      - run: pytest -q
  render:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: sudo apt-get install -y quarto
      - run: pip install -r requirements.txt
      - run: quarto render report_quarto_project.ipynb --to html
'''
print('\nExemple CI:\n')
print(ci_example)


## 9) Reproductibilité et configuration d'exécution

- Fixer seeds: numpy, random, sklearn.
- Exporter `requirements.txt` ou `environment.yml` pour reproduire l'environnement.
- Instructions pour exécuter dans VSCode: ouvrir le kernel Python 3 et exécuter les cellules dans l'ordre.

In [ ]:
# Reproductibilité: fixer seeds
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
print('Seeds fixés à', SEED)

print('\nPour exporter requirements (pip):')
print('  pip freeze > requirements.txt')
print("ou pour conda:\n  conda env export > environment.yml")


## 10) Annexes : fonctions utilitaires et scripts d'analyse

Bloc de fonctions courantes (chargement MongoDB, vérification Change Streams, commandes utiles).

In [ ]:
# Fonctions utilitaires pour ce projet
from pathlib import Path
from pymongo import MongoClient
import os
from dotenv import load_dotenv


def get_mongo_db():
    """Charge .env et retourne une instance db (ou None si échec)"""
    try:
        env_path = Path(__file__).parent / '.env'
    except Exception:
        env_path = Path('.') / '.env'
    try:
        load_dotenv(env_path)
    except Exception:
        pass
    uri = os.getenv('MONGODB_URI', 'mongodb://localhost:27017/')
    db_name = os.getenv('MONGODB_DATABASE', 'Ubereats')
    try:
        client = MongoClient(uri, serverSelectionTimeoutMS=3000)
        # test
        client.server_info()
        return client[db_name]
    except Exception as e:
        print('Erreur connexion MongoDB:', e)
        return None


def check_replica_set():
    db = get_mongo_db()
    if not db:
        print('DB inaccessible')
        return False
    try:
        info = db.client.admin.command('replSetGetStatus')
        print('Replica set status obtenu')
        return True
    except Exception as e:
        print('Replica set non configuré ou erreur:', e)
        return False

print('Fonctions utilitaires prêtes. Exemple: \n- get_mongo_db()\n- check_replica_set()')


### Résumé & prochaines étapes

- Vérifier que MongoDB est configuré en replica set si vous voulez utiliser Change Streams.
- Peupler la base (ex: `py simulate.py --count 500`).
- Lancer le launcher adapté (`launcher_changestreams.py` pour les scripts changestreams ou `launcher_advanced.py` pour les autres).
- Exécuter les tests via `pytest` et rendre le rapport via `quarto render`.

Merci — adaptez les cellules selon vos besoins et exécutez dans l'ordre.